# spectrum demo 2

## get an emission measure file, run mspectrum, save the data and work with the restored data

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import ChiantiPy.core as ch
import ChiantiPy.tools.util as chutil
import ChiantiPy.tools.io as chio
import ChiantiPy.tools.filters as chfilters

In [ ]:
autoreload 3

In [ ]:
matplotlib qt

## get an emission measure file

## these are in XUVTOP/em

In [ ]:
emDir = os.path.join(os.environ['XUVTOP'], 'em')

In [ ]:
emList = os.listdir(emDir)

In [ ]:
for idx, em in enumerate(emList):
    print(' %i  %s'%(idx, em))

## at present, there are only 2 files, a quiet sun EM and an active EM

## I will pick the active region EM

In [ ]:
emAR = chio.emRead(emList[1])

In [ ]:
emAR.keys()

In [ ]:
for idx, temp in enumerate(emAR['temperature']):
    print(' %i  %10.2e  %10.2e  %10.2e'%(idx, temp, emAR['density'][idx], emAR['em'][idx]))

## this is similar to one of the quick start examples but uses the EM distribution

In [ ]:
temp = emAR['temperature']
dens = emAR['density']
em = emAR['em']

In [ ]:
wvlRange = [300.,400]

In [ ]:
dwvl = 0.01
nwvl = int((wvlRange[1] - wvlRange[0])/dwvl + 1)

In [ ]:
wvl = np.linspace(wvlRange[0], wvlRange[1], nwvl)

In [ ]:
' wvl min:  %10.2f  wvl max %10.2f'%(wvl.min(), wvl.max())

### will use mspectrum to use multiple cores/processors

In [ ]:
cores = 4

In [ ]:
sp = ch.mspectrum(temp, dens, wvl, filter = (chfilters.gaussian,30.*dwvl), em = em, doContinuum=0, minAbund=1.e-5,
                  proc=cores)

In [ ]:
sp.spectrumPlot(integrated=True)

## save the calculations to a pickle file

In [ ]:
saveName = 'spectrum_demo_2.pkl'

In [ ]:
sp.saveData(saveName)

In [ ]:
sp.Spectrum.keys()

## restore the calculations from the pickle file

In [ ]:
with open(saveName,'rb') as inpt:
    spectrumDict = pickle.load(inpt)

### examine what is in spectrumDict

In [ ]:
for akey in spectrumDict.keys():
    print(' key:  %s'%(akey))

In [ ]:
spectrumDict['abundanceName']

In [ ]:
spectrumDict['abundAll'].shape

In [ ]:
spectrumDict['defaults'].keys()

In [ ]:
spectrumDict['em'].shape

In [ ]:
spectrumDict['spectrum'].keys()

In [ ]:
spectrumDict['spectrum']['intensity'].shape

In [ ]:
spectrumDict['spectrum']['integrated'].shape

In [ ]:
spectrumDict['nTemp']

In [ ]:
spectrumDict['nDens']

In [ ]:
spectrumDict['nTempDens']

In [ ]:
spectrumDict['spectrum']['wavelength'].min()

In [ ]:
spectrumDict['spectrum']['wavelength'].max()

## make another plot of lines calculated in spectrum/mspectrum

## there is more flexibility when working with the saveData dict

In [ ]:
fs = 10  # fontsize
figsize = [7., 5.]

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

In [ ]:
ax.set_xlim(wvlRange)

In [ ]:
ax.plot(spectrumDict['spectrum']['wavelength'], spectrumDict['spectrum']['integrated'])

In [ ]:
ax.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax.set_ylabel(spectrumDict['ylabel'], fontsize=16)

In [ ]:
fig.tight_layout()

### try a narrower wavelength range

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

In [ ]:
wvlRange = [330.,340.]

In [ ]:
ax.set_xlim(wvlRange)

In [ ]:
ax.plot(spectrumDict['spectrum']['wavelength'], spectrumDict['spectrum']['integrated'])

In [ ]:
ax.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax.set_ylabel(spectrumDict['ylabel'], fontsize=16)

In [ ]:
fig.tight_layout()

## plot the spectrum and label the lines with 2 plots

In [ ]:
figsize = [7., 10.]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1 , figsize=figsize)

In [ ]:
wvlRange = [330.,340.]

In [ ]:
ax1.set_xlim(wvlRange)

In [ ]:
ax2.set_xlim(wvlRange)

In [ ]:
ax1.plot(spectrumDict['spectrum']['wavelength'], spectrumDict['spectrum']['integrated'], 'k', lw=2)

In [ ]:
ylim = ax1.get_ylim()

In [ ]:
ylim

In [ ]:
ax1.set_ylim([0., ylim[1]])

In [ ]:
ax1.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax1.set_ylabel(spectrumDict['ylabel'], fontsize=16)

In [ ]:
ax1.plot(spectrumDict['spectrum']['wavelength'], spectrumDict['spectrum']['integrated'])

In [ ]:
ax1.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax1.set_ylabel(spectrumDict['ylabel'], fontsize=16)

### just plot the labels for the most intense lines, minIntensity could be set to zero

In [ ]:
index = chutil.between(spectrumDict['intensity']['wvl'], wvlRange)

In [ ]:
len(index)

### set to whatever value works for you

In [ ]:
minIntensity = spectrumDict['intensity']['integrated'][index].max()/50.

In [ ]:
' %10.3e'%(minIntensity)

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    integrated = spectrumDict['intensity']['integrated'][idx]
    if integrated > minIntensity:
        ax2.plot([wvl, wvl] , [0., integrated], 'k')

In [ ]:
ylim = ax2.get_ylim()

In [ ]:
ylim

In [ ]:
ax2.set_ylim([0., 1.5*ylim[1]])

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    lbl = spectrumDict['intensity']['ionS'][idx] + ' %7.3f'%(wvl)
    if spectrumDict['intensity']['integrated'][idx] > minIntensity:
        ax2.text(wvl, 1.2*spectrumDict['intensity']['integrated'][idx], lbl, va='bottom', ha='center', rotation='vertical', fontsize=fs)

In [ ]:
ax2.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax2.set_ylabel('Relative', fontsize=16)

In [ ]:
fig.tight_layout()

## plot the spectrum and label the lines in a single plot

In [ ]:
figsize = [7., 7.]

In [ ]:
fig, ax1 = plt.subplots(figsize=figsize)

In [ ]:
wvlRange = [330.,340.]

In [ ]:
ax1.set_xlim(wvlRange)

In [ ]:
ax2.set_xlim(wvlRange)

In [ ]:
ax1.plot(spectrumDict['spectrum']['wavelength'], spectrumDict['spectrum']['integrated'])

In [ ]:
ax1.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax1.set_ylabel(spectrumDict['ylabel'], fontsize=16)

In [ ]:
ax1.plot(spectrumDict['spectrum']['wavelength'], spectrumDict['spectrum']['integrated'])

In [ ]:
ax1.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax1.set_ylabel(spectrumDict['ylabel'], fontsize=16)

### just plot the labels for the most intense lines, minIntensity could be set to zero

In [ ]:
intIndex = chutil.between(spectrumDict['intensity']['wvl'], wvlRange)

In [ ]:
len(intIndex)

In [ ]:
spIndex = chutil.between(spectrumDict['spectrum']['wavelength'], wvlRange)

In [ ]:
len(spIndex)

In [ ]:
minIntensity1 = spectrumDict['spectrum']['integrated'][spIndex].max()/50.

In [ ]:
' %10.3e'%(minIntensity1)

In [ ]:
minIntensity2 = spectrumDict['intensity']['integrated'][intIndex].max()/20.

In [ ]:
' %10.3e'%(minIntensity2)

In [ ]:
maxIntensity1 = spectrumDict['spectrum']['integrated'][spIndex].max()

In [ ]:
' %10.3e'%(maxIntensity1)

In [ ]:
maxIntensity2 = spectrumDict['intensity']['integrated'][intIndex].max()

In [ ]:
' %10.3e'%(maxIntensity2)

In [ ]:
scaleFactor = 1.1*maxIntensity1/maxIntensity2

In [ ]:
scaleFactor

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    integrated = spectrumDict['intensity']['integrated'][idx]
    if integrated > minIntensity2:
        ax1.plot([wvl, wvl] , [0., scaleFactor*integrated], 'k')

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    lbl = spectrumDict['intensity']['ionS'][idx] + ' %7.3f'%(wvl)
    if spectrumDict['intensity']['integrated'][idx] > minIntensity1:
        ax1.text(wvl, scaleFactor*spectrumDict['intensity']['integrated'][idx], lbl, va='bottom', ha='center', rotation='vertical', fontsize=fs)

In [ ]:
ylim = ax1.get_ylim()

In [ ]:
ylim

In [ ]:
ax1.set_ylim([0., scaleFactor*ylim[1]])

In [ ]:
fig.tight_layout()